In [15]:
from pathlib import Path
from medkit.core.text import TextDocument
from medkit.text.segmentation import SentenceTokenizer
from medkit.text.ner import RegexpMatcher, RegexpMatcherRule , RegexpMatcherNormalization
from medkit.text.context import NegationDetector, NegationDetectorRule
from medkit.text.segmentation import SyntagmaTokenizer
from medkit.text.context import FamilyDetector
from unidecode import unidecode
import os
import re
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [16]:
def preprocessing(text):
    
    # Convertir les caractères spéciaux spécifiques avant la conversion en ASCII
    text = re.sub(r'n°', 'numero', text)  # Remplace "n°" par "numero"
    text = re.sub(r'/d°', 'deg', text)  # Remplace "/d°" par "deg"

    # Convertir le texte en ASCII
    ascii_text = unidecode(text)  # Convertit les caractères Unicode en ASCII

    # Normaliser les espaces en remplaçant les espaces multiples par un seul espace
    ascii_text = re.sub(r'\s+', ' ', ascii_text)  # Remplace plusieurs espaces par un seul espace
    return ascii_text

In [17]:
"""
La fonction statut_extraction extrait les informations de statut de tabagisme, 
statut marital et statut d'alcool à partir d'un dictionnaire dico en parcourant les 
annotations, en calculant les proportions et en choisissant les statuts appropriés. 
Elle renvoie les statuts extraits.
"""

def statut_extraction_tabac(doc):

    ## Initialisation
    statut = "UNKNOWN"

    # Nombre d'entités trouvées
    n_oui = 0
    n_non = 0

    # Proportions calculées à partir du nombre d'entités trouvées
    p_neg = 0
    p_pos = 0

    # Parcours des annotations pour analyser chaque annotation trouvée
    for ann in doc.anns:
        if ann.label == "statut_tabagisme":
            for attr in ann.attrs:
                if attr.label == "is_negated":
                    value_is_negated = attr.value
                elif attr.label == "other_detected":
                    value_other_detected = attr.value

            # Si l'entité trouvée ne concerne pas le patient (other_detected == True),
            # on passe à l'annotation suivante
            if value_other_detected == True:
                continue
            else:
                # Si c'est une négation, on incrémente n_non_tabac de 1
                if value_is_negated == True:
                    n_non += 1
                # Sinon, on incrémente n_oui_tabac de 1
                else:
                    n_oui += 1

    if len(doc.anns) != 0:
        # Calcul des proportions de tabagisme
        p_neg = n_non / len(doc.anns)
        p_pos = n_oui / len(doc.anns)

    ## Choix du statut en fonction des proportions calculées
    if p_pos > p_neg:
        statut = "FUMEUR"
    elif p_pos < p_neg:
        statut = "NON-FUMEUR"


    return statut

In [45]:
"""
La fonction statut_extraction extrait les informations de statut d'alcool à partir d'un dictionnaire dico en
parcourant les  annotations, en calculant les proportions et en choisissant les statuts appropriés. 
Elle renvoie les statuts extraits.
"""

def statut_extraction_alcool(doc):

    ## Initialisation
    statut = "UNKNOWN"

    # Nombre d'entités trouvées
    n_oui = 0
    n_non = 0

    # Proportions calculées à partir du nombre d'entités trouvées
    p_neg = 0
    p_pos = 0

    # Parcours des annotations pour analyser chaque annotation trouvée
    for ann in doc.anns:
        if ann.label == "statut_alcool":
            for attr in ann.attrs:
                if attr.label == "is_negated":
                    value_is_negated = attr.value
                elif attr.label == "other_detected":
                    value_other_detected = attr.value

            # Si l'entité trouvée ne concerne pas le patient (other_detected == True),
            # on passe à l'annotation suivante
            if value_other_detected == True:
                continue
            else:
                # Si c'est une négation, on incrémente n_non_tabac de 1
                if value_is_negated == True:
                    n_non += 1
                    
                # Sinon, on incrémente n_oui_tabac de 1
                else:
                    n_oui += 1

    if len(doc.anns) != 0:
        # Calcul des proportions de tabagisme
        p_neg = n_non / len(doc.anns)
        p_pos = n_oui / len(doc.anns)
    ## Choix du statut en fonction des proportions calculées
    if p_pos > p_neg:
        statut = "ALCOOLIQUE"
    elif p_pos < p_neg:
        statut = "NON-AlCOOLIQUE"


    return statut


In [46]:
def statut_extraction_situation_familiale(doc):

    ## Initialisation
    statut_marital = "UNKNOWN"

    # On parcourt le dico pour analyser chaque annotation trouvé
    for ann in doc.anns:
        if ann.label== "statut_marital":
            for attr in ann.attrs:
                if attr.label == "is_negated":
                    value_is_negated = attr.value
                elif attr.label == "other_detected":
                    value_other_detected = attr.value

            # Si l'entité trouvée ne concerne pas le patient (other_detected == True),
            # on passe à l'annotation suivante
            if value_other_detected == True:
                continue
            else:
                
                statut_marital = ann.text
                
                 ## NORMALISATION: Seul, pas seul et inconnu
                if re.search(r"\bmarie[e]?\b", statut_marital):
                    statut_marital = "PAS SEUL"
                elif re.search(r"\bcelibataire\b", statut_marital):
                    statut_marital = "SEUL"
                elif re.search(r"\bdivorce[e]?\b", statut_marital):
                    statut_marital = "SEUL"
                elif re.search(r"\bveuf\b", statut_marital):
                    statut_marital = "SEUL"
                elif re.search(r"\bveuve\b", statut_marital):
                    statut_marital = "SEUL"
                elif re.search(r"\bpacse[e][s]?\b", statut_marital):
                    statut_marital = "PAS SEUL"
                elif re.search(r"\bconcubinage\b", statut_marital):
                    statut_marital = "PAS SEUL"
                elif re.search(r"\b(vit|habite)\sseul(e)?\b", statut_marital):
                    statut_marital = "SEUL"
                    
                # Si il y a une négation
                if value_is_negated == True:
                    # On inverse statut_marital
                    if statut_marital == "SEUL":
                        statut_marital = "PAS SEUL"
                    else:
                        statut_marital = "SEUL"

    return statut_marital  

In [47]:
def clinical_case_recovery(output_folder):
    # On récupère tous les fichiers texte dans le dossier
    txt_files = [f for f in os.listdir(output_folder) if f.endswith('.txt')]

    # On trie les fichiers par ordre alphabétique.
    txt_files_sorted = sorted(txt_files)
    textes = {}  # dictionnaire de tous les cas cliniques

    # On ouvre et on extrait les textes dans textes
    for i in range(717):
        file_path = os.path.join(output_folder, txt_files_sorted[i])
        with open(file_path, 'r') as f:
            text = f.read()
        textes[txt_files_sorted[i]] = text
    return textes

## EXTRACTION STATUT TABAGIQUE

In [48]:
# Listes pour stocker les données
data_fumeur = []
data_non_fumeur = []
data_unknown = []

### QUELQUES REGEX RECHERCHE ENTITES

regexp_rules = [

    RegexpMatcherRule(regexp=r"\bcigare(tte)?[s]?\b", label="statut_tabagisme", exclusion_regexp ="en bout de cigare"),
    RegexpMatcherRule(regexp=r"\bfume(e)?[s]?\b", label="statut_tabagisme"),
    RegexpMatcherRule(regexp=r"\btaba(c|gisme|gique)\b", label="statut_tabagisme"),
    RegexpMatcherRule(
    regexp=r"fumeur|fumeuse|paquet.{0,5}?ann[ée].?",
    label="statut_tabagisme",
    unicode_sensitive=True),
]

### QUELQUES REGEX NEGATION

neg_rules = [       

NegationDetectorRule(regexp=r"\bne\s*(semble|consomme|prend)\s*pas"),
NegationDetectorRule(regexp=r"jamais"),
NegationDetectorRule(regexp=r"arret"),
NegationDetectorRule(regexp=r"ancien"),
NegationDetectorRule(regexp=r"\bni\b"),
NegationDetectorRule(regexp=r"\bnon\b"),
#NegationDetectorRule(regexp=r"\b(ni|non)\/sfumeu(r|se)"),
NegationDetectorRule(regexp=r"\bne/s*fume/s*pas\b"),
NegationDetectorRule(regexp=r"non\\s+fumeur|tabac\\s+non|tabagisme\\s+non|Tabac\\s*[=:]?\\s*0|tabagi(sm|qu)e\\s+sevr|(pas|ni|ou)\\s+de\\s+(consommation\\s+de\\s+)?taba|pas\\s+d\\'intoxication\\s+tabagi|0 tabac",
                    unicode_sensitive=True),

]

In [49]:
def extraction_finale_tabac(clinical_case_repo):
    # On charge les cas cliniques dans un dico{nom fichier}=cas clinique
    clinical_cases_dico = clinical_case_recovery(clinical_case_repo)

    for fichier, clinical_case in clinical_cases_dico.items():
        clinical_case = preprocessing(clinical_case)
        doc = TextDocument(text=clinical_case)

        ## On sépare le texte en phrases
        sent_tokenizer = SentenceTokenizer(
            output_label="sentence",
            punct_chars=[".", "?", "!"],
        )
        sentences = sent_tokenizer.run([doc.raw_segment])

        ## On sépare les phrases en syntagmas
        synt_tokenizer = SyntagmaTokenizer(
            output_label="sentence",
            separators=[r"\bmais\b", r"\bet\b"],
        )
        syntagmas = synt_tokenizer.run(sentences)


        # Création de l'objet neg detector
        neg_detector = NegationDetector(output_label="is_negated", rules=neg_rules)

        # On applique neg detector aux syntagmas
        neg_detector.run(syntagmas)

        # On applique family detector aux syntagmas
        family_detector = FamilyDetector(output_label='other_detected')
        family_detector.run(syntagmas)

        # CREATION OF ENTITIES
        regexp_matcher = RegexpMatcher(rules=regexp_rules, attrs_to_copy=["is_negated", "other_detected"])

        entities = regexp_matcher.run(syntagmas)

        for entity in entities:
            doc.anns.add((entity))

        statut_tabagisme = statut_extraction_tabac(doc)


        # Remplissage de data
        if statut_tabagisme == "FUMEUR":
            data_fumeur.append([fichier, clinical_case, statut_tabagisme])
        elif statut_tabagisme == "NON-FUMEUR":
            data_non_fumeur.append([fichier, clinical_case, statut_tabagisme])
        else:
            if len(data_unknown) < 17:
                data_unknown.append([fichier, clinical_case, statut_tabagisme])


    df_fumeur = pd.DataFrame(data_fumeur, columns=["nom fichier", "cas clinique","statut tabagisme"])


    df_non_fumeur = pd.DataFrame(data_non_fumeur, columns=["nom fichier", "cas clinique","statut tabagisme"])
    df_unknown = pd.DataFrame(data_unknown, columns=["nom fichier", "cas clinique","statut tabagisme"])
    combined_df = pd.concat([df_fumeur, df_non_fumeur, df_unknown], ignore_index=True)
    # On mélange 
    combined_df_tabac = combined_df.sample(frac=1, random_state=42)
    combined_df.shape
    combined_df_tabac['statut tabagisme'].value_counts()
    
    return combined_df_tabac

In [50]:
df_tabac_1 = extraction_finale_tabac("clinical_case1")
df_tabac_1['statut tabagisme'].value_counts()

statut tabagisme
FUMEUR        23
UNKNOWN       17
NON-FUMEUR    15
Name: count, dtype: int64

## EXTRACTION STATUT ALCOOL

In [51]:
# Listes pour stocker les données
data_oui = []
data_non = []
data_unknown = []

### QUELQUES REGEX RECHERCHE ENTITES

regexp_rules= [

    RegexpMatcherRule(regexp=r"\balcool\b", label="statut_alcool"),
    RegexpMatcherRule(regexp=r"\bboit\b", label="statut_alcool"),
    RegexpMatcherRule(regexp=r"\balcoolique\b", label="statut_alcool"),
    RegexpMatcherRule(regexp=r"\bdependance\s*alcool\b", label="statut_alcool"),
    RegexpMatcherRule(regexp=r"\balcoolisme\b", label="statut_alcool"),

]

### QUELQUES REGEX NEGATION

neg_rules = [

    NegationDetectorRule(regexp=r"ne\s*boit\s*pas"),
    NegationDetectorRule(regexp=r"\bne/s*consomme/s*pas\b"),
    NegationDetectorRule(regexp=r"\bni/s*alcool\b"),
    NegationDetectorRule(regexp=r"\bpas\b"),
    NegationDetectorRule(regexp=r"\becarte\b"),
]

In [52]:

def extration_finale_alcool(clinical_case_repo):
    # On charge les cas cliniques dans un dico{nom fichier}=cas clinique
    clinical_cases_dico = clinical_case_recovery("clinical_case2")

    for fichier, clinical_case in clinical_cases_dico.items():
        clinical_case = preprocessing(clinical_case)
        doc = TextDocument(text=clinical_case)

        ## On sépare le texte en phrases
        sent_tokenizer = SentenceTokenizer(
            output_label="sentence",
            punct_chars=[".", "?", "!"],
        )
        sentences = sent_tokenizer.run([doc.raw_segment])

        ## On sépare les phrases en syntagmas
        synt_tokenizer = SyntagmaTokenizer(
            output_label="sentence",
            separators=[r"\bmais\b", r"\bet\b"],
        )
        syntagmas = synt_tokenizer.run(sentences)


        # Création de l'objet neg detector
        neg_detector = NegationDetector(output_label="is_negated", rules=neg_rules)

        # On applique neg detector aux syntagmas
        neg_detector.run(syntagmas)

        # On applique family detector aux syntagmas
        family_detector = FamilyDetector(output_label='other_detected')
        family_detector.run(syntagmas)

        # CREATION OF ENTITIES
        regexp_matcher = RegexpMatcher(rules=regexp_rules, attrs_to_copy=["is_negated", "other_detected"])

        entities = regexp_matcher.run(syntagmas)

        for entity in entities:
            doc.anns.add((entity))

        statut = statut_extraction_alcool(doc)


        # Remplissage de data
        if statut == "ALCOOLIQUE":
            data_oui.append([fichier, clinical_case, statut])
        elif statut == "NON-AlCOOLIQUE":
            data_non.append([fichier, clinical_case, statut])
        else:
            if len(data_unknown) < 17:
                data_unknown.append([fichier, clinical_case, statut])


    df_oui = pd.DataFrame(data_oui, columns=["nom fichier", "cas clinique","statut alcool"])
    df_non = pd.DataFrame(data_non, columns=["nom fichier", "cas clinique","statut alcool"])
    df_unknown = pd.DataFrame(data_unknown, columns=["nom fichier", "cas clinique","statut alcool"])
    combined_df = pd.concat([df_oui, df_non, df_unknown], ignore_index=True)
    # On mélange 
    combined_df_alcool = combined_df.sample(frac=1, random_state=42)
    
    return combined_df_alcool

In [54]:
df_alcool_1 = extration_finale_alcool("clinical_case1")
df_alcool_1['statut alcool'].value_counts()

statut alcool
ALCOOLIQUE        40
UNKNOWN           17
NON-AlCOOLIQUE    12
Name: count, dtype: int64

## EXTRACTION STATUT FAMILIAL

In [ ]:
# Listes pour stocker les données
data_oui = []
data_non = []
data_unknown = []

### QUELQUES REGEX RECHERCHE ENTITES

regexp_rules= [

    RegexpMatcherRule(regexp=r"\bmarie[e]?\b", label="statut_marital"),
    RegexpMatcherRule(regexp=r"\bcelibataire\b", label="statut_marital"),
    RegexpMatcherRule(regexp=r"\bdivorce[e]?\b", label="statut_marital"),
    RegexpMatcherRule(regexp=r"\bveuf\b", label="statut_marital"),
    RegexpMatcherRule(regexp=r"\bveuve\b", label="statut_marital"),
    RegexpMatcherRule(regexp=r"\bpacse[e][s]?\b", label="statut_marital"),
    RegexpMatcherRule(regexp=r"\bconcubinage\b", label="statut_marital"),
    RegexpMatcherRule(regexp=r"\b(vit|habite)\sseul(e)?\b", label="statut_marital"),

]

### QUELQUES REGEX NEGATION

neg_rules = [

    NegationDetectorRule(regexp=r"\bn'est pas\b"),
    NegationDetectorRule(regexp=r"\bne vit pas\b"),
    NegationDetectorRule(regexp=r"\bn'habite pas\b"),
    NegationDetectorRule(regexp=r"\bni\b"),
]

In [61]:
def extraction_finale_statut_familial(clinical_case_repo):

    # On charge les cas cliniques dans un dico{nom fichier}=cas clinique
    clinical_cases_dico = clinical_case_recovery("clinical_case2")

    for fichier, clinical_case in clinical_cases_dico.items():
        clinical_case = preprocessing(clinical_case)
        doc = TextDocument(text=clinical_case)

        ## On sépare le texte en phrases
        sent_tokenizer = SentenceTokenizer(
            output_label="sentence",
            punct_chars=[".", "?", "!"],
        )
        sentences = sent_tokenizer.run([doc.raw_segment])

        ## On sépare les phrases en syntagmas
        synt_tokenizer = SyntagmaTokenizer(
            output_label="sentence",
            separators=[r"\bmais\b", r"\bet\b"],
        )
        syntagmas = synt_tokenizer.run(sentences)


        # Création de l'objet neg detector
        neg_detector = NegationDetector(output_label="is_negated", rules=neg_rules)

        # On applique neg detector aux syntagmas
        neg_detector.run(syntagmas)

        # On applique family detector aux syntagmas
        family_detector = FamilyDetector(output_label='other_detected')
        family_detector.run(syntagmas)

        # CREATION OF ENTITIES
        regexp_matcher = RegexpMatcher(rules=regexp_rules, attrs_to_copy=["is_negated", "other_detected"])

        entities = regexp_matcher.run(syntagmas)

        for entity in entities:
            doc.anns.add((entity))

        statut = statut_extraction_situation_familiale(doc)


        # Remplissage de data
        if statut == "SEUL":
            data_oui.append([fichier, clinical_case, statut])
        elif statut == "PAS SEUL":
            data_non.append([fichier, clinical_case, statut])
        else:
            if len(data_unknown) < 17:
                data_unknown.append([fichier, clinical_case, statut])


    df_oui = pd.DataFrame(data_oui, columns=["nom fichier", "cas clinique","statut familial"])
    df_non = pd.DataFrame(data_non, columns=["nom fichier", "cas clinique","statut familial"])
    df_unknown = pd.DataFrame(data_unknown, columns=["nom fichier", "cas clinique","statut familial"])
    combined_df = pd.concat([df_oui, df_non, df_unknown], ignore_index=True)
    # On mélange 
    combined_df_familial = combined_df.sample(frac=1, random_state=42)
    return combined_df_familial

In [63]:
df_familial_1 = extraction_finale_statut_familial("clinical_case2")
df_familial_1['statut familial'].value_counts()

statut familial
ALCOOLIQUE        40
UNKNOWN           17
NON-AlCOOLIQUE    12
Name: count, dtype: int64

In [87]:
pd.set_option('display.max_colwidth', None)
combined_df


,nom fichier,cas clinique,statut tabagisme
39,EN101019.txt,"Madame H.A est une patiente de 32 ans, primipare, primigeste, sans antecedents particuliers. Elle a ete admise aux urgences de gynecologie-obstetrique du centre hospitalier universitaire de Rabat pour un etat de choc apres un accouchement dystocique dans une maternite peripherique. Des expressions abdominales ainsi qu'une extraction instrumentale ont ete realisee donnant naissance a un nouveau-ne pesant 3050g et dont l'Apgar a ete estime respectivement a 8/10 et 10/10 a la premiere et a la dixieme minute. L'accouchement s'est complique d'une hemorragie de la delivrance jugule par la perfusion d'ocytocique et un massage uterin. Le saignement s'est arrete, mais l'etat hemodynamique de la patiente s'altererait, d'ou son transfert dans notre formation. A l'admission on trouvait une patiente obnubile, polypneique et tachycarde a 120 bat/min, la tension arterielle etait a 70/40 mmHg. Apres mise en condition notamment une oxygenotherapie et un remplissage par serum sale, un examen general a pu etre realise montrant une paleur cutaneo-muqueuse generalisee. L'abdomen etait legerement distendu avec une sensibilite de l'hypochondre gauche. La retraction uterine etait satisfaisante sans saignement exteriorise. Une revision uterine sous couverture antibiotique a ete realisee faisant eliminer une rupture uterine. Un examen sous valve de la filiere genitale etait normal. Le bilan paraclinique objectivait une anemie a 4,5g/dl d'Hemoglobine necessitant la transfusion de 6 culots globulaires. L'echographie abdominopelvienne revelait un epanchement de moyenne abondance, une rate augmentee de volume tres heterogene avec fracture splenique au niveau du pole superieur, le foie et les reins etaient sans anomalies. Le balayage echographique sur l'uterus retrouvait une ligne de vacuite fine sans image de solution de continuite des parois uterines. Une decision pluridisciplinaire d'une laparotomie d'urgence a ete prise, l'equipe se composait de chirurgien obstetricien, un chirurgien visceral et un reanimateur. Une incision mediane a cheval sur l'ombilic a ete realisee. A l'ouverture on retrouvait un hemo-peritoine d'environ 3 litres de sang incoagulable avec des caillots. L'exploration du pelvis notait l'integrite de l'uterus, des annexes et des parametres. L'exploration de la loge splenique decouvrait la rupture du pedicule vasculaire de la rate et une dilaceration du parenchyme splenique avec suffusion hemorragique. Une splenectomie totale a ete realisee apres ligatures des vaisseaux spleniques, suivie d'un drainage de la loge splenique et fermetures plan par plan. La patiente etait stable sur le plan hemodynamique et fut transferee par la suite au service de reanimation ou elle a etait nouvellement transfusee. Les suites operatoires etaient simples et les controles post-operatoires etaient normaux. La patiente est declaree sortante dix jours apres l'intervention.",UNKNOWN
25,EN101713.txt,"Il s'agissait d'un patient de 30 ans, sans antecedents pathologiques particuliers, notamment non tabagique, non alcoolique. Ce patient s'est presente aux urgences pour une tumefaction basi-cervicale anterieure evoluant depuis 3 mois, indolore, associee a une dysphonie, une dysphagie, une dyspnee d'aggravation progressive et un amaigrissement non chiffre, le tout evoluant dans un contexte d'alteration de l'etat general. A l'examen physique il etait apyretique a 37degC (avec une notion de prise anterieure d'antipyretiques) et presentait une masse basi cervicale paramediane gauche mobile, ascensionnant a la deglutition, de consistance molle, sans signes inflammatoires en regard, associee a un leger tirage sus sternal. L'hemogramme objectivait une hyperleucocytose a 11800/uL avec 77% de polynucleaires neutrophiles (PNN). La proteine C reactive (CRP) initiale etait elevee a 354,1 mg/l. La TSH ultrasensible etait basse, a 0,02 UI/mL. La serologie HIV etait negative. L'echographie cervicale mettait en evidence un

In [34]:
### QUELQUES REGEX RECHERCHE ENTITES

regexp_rules_tabac = [

    RegexpMatcherRule(regexp=r"\bcigare(tte)?[s]?\b", label="statut_tabagisme"),
    RegexpMatcherRule(regexp=r"\bfume(e)?[s]?\b", label="statut_tabagisme"),
    RegexpMatcherRule(regexp=r"\btaba(c|gisme|gique)\b", label="statut_tabagisme"),
    RegexpMatcherRule(
    regexp=r"fumeur|fumeuse|paquet.{0,5}?ann[ée].?",
    label="statut_tabagisme",
    unicode_sensitive=True),
]

regexp_rules_alcool= [

    RegexpMatcherRule(regexp=r"\balcool\b", label="statut_alcool"),
    RegexpMatcherRule(regexp=r"\bboit\b", label="statut_alcool"),
    RegexpMatcherRule(regexp=r"\balcoolique\b", label="statut_alcool"),
    RegexpMatcherRule(regexp=r"\bdependance\s*alcool\b", label="statut_alcool"),
    RegexpMatcherRule(regexp=r"\balcoolisme\b", label="statut_alcool"),

]

regexp_rules_situation_familiale= [

    RegexpMatcherRule(regexp=r"\bmarie[e]?\b", label="statut_marital"),
    RegexpMatcherRule(regexp=r"\bcelibataire\b", label="statut_marital"),
    RegexpMatcherRule(regexp=r"\bdivorce[e]?\b", label="statut_marital"),
    RegexpMatcherRule(regexp=r"\bveuf\b", label="statut_marital"),
    RegexpMatcherRule(regexp=r"\bveuve\b", label="statut_marital"),
    RegexpMatcherRule(regexp=r"\bpacse[e][s]?\b", label="statut_marital"),
    RegexpMatcherRule(regexp=r"\bconcubinage\b", label="statut_marital"),
    RegexpMatcherRule(regexp=r"\b(vit|habite)\sseul(e)?\b", label="statut_marital"),

]

# On convertit les règles avec RegexpMatcher
regexp_matcher_tabac = RegexpMatcher(rules=regexp_rules_tabac)
regexp_matcher_alcool = RegexpMatcher(rules=regexp_rules_alcool)
regexp_matcher_situation_familiale = RegexpMatcher(rules=regexp_rules_situation_familiale)

### QUELQUES REGEX NEGATION

neg_rules_tabac = [       

NegationDetectorRule(regexp=r"\bne\s*(semble|consomme|prend)\s*pas"),
NegationDetectorRule(regexp=r"jamais"),
NegationDetectorRule(regexp=r"arret"),
NegationDetectorRule(regexp=r"ancien"),
NegationDetectorRule(regexp=r"\bnon\s*tabagi(que|sme)"),
NegationDetectorRule(regexp=r"\b(ni|non)\/sfumeu(r|se)"),
NegationDetectorRule(regexp=r"\bne/s*fume/s*pas\b"),
NegationDetectorRule(regexp=r"non\\s+fumeur|tabac\\s+non|tabagisme\\s+non|Tabac\\s*[=:]?\\s*0|tabagi(sm|qu)e\\s+sevr|(pas|ni|ou)\\s+de\\s+(consommation\\s+de\\s+)?taba|pas\\s+d\\'intoxication\\s+tabagi|0 tabac",
                    unicode_sensitive=True),

]

neg_rules_alcool = [

    NegationDetectorRule(regexp=r"ne\s*boit\s*pas"),
    NegationDetectorRule(regexp=r"\bne/s*consomme/s*pas\b"),
    NegationDetectorRule(regexp=r"\bni/s*alcool\b"),
]

neg_rules_situation_familiale = [

    NegationDetectorRule(regexp=r"\bn'est pas\b"),
    NegationDetectorRule(regexp=r"\bne vit pas\b"),
    NegationDetectorRule(regexp=r"\bn'habite pas\b"),
    NegationDetectorRule(regexp=r"\bni\b"),
]

   
# Création de l'objet neg detector
neg_detector_tabac = NegationDetector(output_label="is_negated", rules=neg_rules_tabac)
neg_detector_alcool = NegationDetector(output_label="is_negated", rules=neg_rules_alcool)
neg_detector_situation_familiale = NegationDetector(output_label="is_negated", rules=neg_rules_situation_familiale)



# On charge les cas cliniques dans un dico{nom fichier}=cas clinique
clinical_cases_dico = clinical_case_recovery("clinical_case1")

for fichier, clinical_case in clinical_cases_dico.items():
    clinical_case = preprocessing(clinical_case)
    doc = TextDocument(text=clinical_case)
    
    ## On sépare le texte en phrases
    sent_tokenizer = SentenceTokenizer(
        output_label="sentence",
        punct_chars=[".", "?", "!"],
    )
    sentences = sent_tokenizer.run([doc.raw_segment])

    ## On sépare les phrases en syntagmas
    synt_tokenizer = SyntagmaTokenizer(
        output_label="sentence",
        separators=[r"\bmais\b", r"\bet\b"],
    )
    syntagmas = synt_tokenizer.run(sentences)
    
    list_neg_detector = [neg_detector_tabac,neg_detector_alcool,neg_detector_situation_familiale]
    
    for neg_detector in list_neg_detector:
        
        # On applique neg detector aux syntagmas
        neg_detector.run(syntagmas)

        # On applique family detector aux syntagmas
        family_detector = FamilyDetector(output_label='other_detected')
        family_detector.run(syntagmas)

        # CREATION OF ENTITIES
        regexp_matcher = RegexpMatcher(rules=regexp_rules_tabac + regexp_rules_alcool + regexp_rules_situation_familiale,
                                       attrs_to_copy=["is_negated", "other_detected"])

        entities = regexp_matcher.run(syntagmas)
        for entity in entities:
            doc.anns.add((entity))
        
        if neg_detector == neg_detector_tabac:
            statut_tabagisme = statut_extraction_tabac(doc)
        elif neg_detector == neg_detector_alcool:
            statut_alcool = statut_extraction_alcool(doc)
        else:
            statut_familial = statut_extraction_situation_familiale(doc)

    if statut_tabagisme == "FUMEUR":
        print(cinical_case)
        print(statut_tabagisme)
        print("\n")



Un cycliste âgé de 17 ans a été victime d’un accident de vélo avec réception du guidon au niveau de l’épigastre occasionnant des épigastralgies. A l’admission, l’abdomen était sensible, la lipasémie était à 10 fois la normale. La TDM montrait une fracture isthmique du pancréas avec une pancréatite stade E de Balthazar [6] (figure 1), le Wirsung traversant la zone d’hyperdensité, sa rupture ne pouvait être écartée (Classe III Lucas [7]). L’attitude était le traitement médical de la pancréatite, avec surveillance clinique et échographique. L’évolution était favorable sans séquelles.

FUMEUR


Un cycliste âgé de 17 ans a été victime d’un accident de vélo avec réception du guidon au niveau de l’épigastre occasionnant des épigastralgies. A l’admission, l’abdomen était sensible, la lipasémie était à 10 fois la normale. La TDM montrait une fracture isthmique du pancréas avec une pancréatite stade E de Balthazar [6] (figure 1), le Wirsung traversant la zone d’hyperdensité, sa rupture ne pouvai

Un cycliste âgé de 17 ans a été victime d’un accident de vélo avec réception du guidon au niveau de l’épigastre occasionnant des épigastralgies. A l’admission, l’abdomen était sensible, la lipasémie était à 10 fois la normale. La TDM montrait une fracture isthmique du pancréas avec une pancréatite stade E de Balthazar [6] (figure 1), le Wirsung traversant la zone d’hyperdensité, sa rupture ne pouvait être écartée (Classe III Lucas [7]). L’attitude était le traitement médical de la pancréatite, avec surveillance clinique et échographique. L’évolution était favorable sans séquelles.

FUMEUR


Un cycliste âgé de 17 ans a été victime d’un accident de vélo avec réception du guidon au niveau de l’épigastre occasionnant des épigastralgies. A l’admission, l’abdomen était sensible, la lipasémie était à 10 fois la normale. La TDM montrait une fracture isthmique du pancréas avec une pancréatite stade E de Balthazar [6] (figure 1), le Wirsung traversant la zone d’hyperdensité, sa rupture ne pouvai